<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/4.9_소프트맥스_실습.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 소프트맥스 (다중 분류 로지스틱 회귀 모델)
M개의 입력을 받아 N개의 클래스로 출력하는 로지스틱 회귀 모델을 케라스로 구현해보도록 하겠습니다.  
보통 다중 분류 로지스틱 회귀 모델을 소프트맥스(softmax)라고 부릅니다.  
케라스에서 제공하는 MNIST 데이터를 사용하여 숫자를 0에서부터 9까지 분류해보도록 하겠습니다.  

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

MNIST 손글씨 데이터를 다운로드 받아서 변수에 저장합니다.

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


손글씨 데이터(X_train, X_test)가 가로 28픽셀, 세로 28픽셀로 구성된 것을 확인할 수 있습니다.  
학습에 사용될 X_train은 총 60000개의 데이터, 테스트에 사용될 X_test는 총 10000개의 데이터가 있습니다.

In [4]:
print("train data (count, row, column) : " + str(X_train.shape) )
print("test data  (count, row, column) : " + str(X_test.shape) )

train data (count, row, column) : (60000, 28, 28)
test data  (count, row, column) : (10000, 28, 28)


학습 데이터의 하나를 샘플로 보도록 하겠습니다. 아래 보시는 것처럼, 각각의 픽셀은 0부터 255까지의 값을 가지고 있습니다.

In [5]:
print(X_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

모델 학습 시작에 앞서, 데이터를 정규화합니다.  
정규화는 입력값을 0부터 1의 값으로 변경하게 됩니다.  
정규화된 입력값은 경사하강법으로 모델 학습 시, 보다 쉽고 빠르게 최적의 W,B를 찾는 데 도움을 줍니다.

In [6]:
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train /= 255 
X_test /= 255

아래의 명령어를 통해 정규화된 데이터를 확인할 수 있습니다.

In [7]:
print(X_train[0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

y_train, y_test는 손글씨 데이터 (28*28 픽셀 데이터)에 해당하는 숫자를 나타냅니다.  
y_train은 총 6만개, y_test는 총 1만개의 숫자를 가지고 있습니다.

In [8]:
print("train target (count) : " + str(y_train.shape) )
print("test target  (count) : " + str(y_test.shape) )

train target (count) : (60000,)
test target  (count) : (10000,)


아래의 코드를 실행하여, y_train과 y_test에서 샘플로 숫자를 출력해봅니다.

In [9]:
print("sample from train : " + str(y_train[0]) )
print("sample from test : " + str(y_test[0]) )

sample from train : 5
sample from test : 7


이번 실습에서는 28*28 픽셀의 지역적인 정보를 사용하지 않고, 단순히 정규화된 입력값만을 가지고,  
숫자 분류를 할 것이기 때문에, 행과 열의 구분 없이, 단순히 784 길이의 배열로 데이터를 단순화시킵니다.

In [10]:
input_dim = 784 #28*28 
X_train = X_train.reshape(60000, input_dim) 
X_test = X_test.reshape(10000, input_dim) 

아래의 명령어를 실행하여, 현재 우리의 데이터가 2차원 데이터가 아닌 단순한 1차원 데이터로 변경된 것을 확인할 수 있습니다.

In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


학습 시, y값과의 cross entropy를 측정해야하므로, 아래의 코드를 실행하여 y를 one hot encoding으로 변환시켜줍니다.

In [12]:
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

아래 코드를 실행하여, 5였던 값이, one hot encoding으로 변환되어,  
클래스 갯수만큼의 길이를 갖는 벡터로 변경이 되었고, 5에 해당되는 인덱스의 값만 1인 것을 확인할 수 있습니다.

In [13]:
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


케라스의 Sequential()을 사용하여 간단하게 소프트맥스를 구현할 수 있습니다.  
총 784개(28*28)의 입력을 받아서, 10개의 시그모이드 값을 출력하는 모델을 아래의 코드를 실행하여 구현합니다.

In [14]:
model = Sequential() 
model.add(Dense(input_dim=input_dim, units = 10, activation='softmax'))

모델의 학습을 진행합니다.  
10개의 클래스로 분류할 것이기 때문에, categorical_crossentropy를 비용함수로 사용한 경사하강법으로 최적의 W와 biases를 학습합니다.

In [15]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
model.fit(X_train, y_train, batch_size=2048, epochs=100, verbose=0) 

테스트를 진행하여, 정확도를 측정합니다.

In [17]:
score = model.evaluate(X_test, y_test) 
print('Test accuracy:', score[1])

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test accuracy: 0.8908


아래의 코드를 실행하여, 소프트맥스 모델의 구조를 쉽게 시각화 할 수 있습니다.  
총 10개의 로지스틱회귀가 있고, 각 로지스틱회귀는 784개의 weight와 1개의 bias를 갖고 있기 때문에,  
총 7850 (785*10)개의 Param이 있는 것을 보실 수 있습니다.

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


첫번째 레이어에 존재하는 w1, w2,...,w784, b1, b2,..., b10은 아래의 명령어로 확인하실 수 있습니다.

In [19]:
model.layers[0].weights

[<tf.Variable 'dense/kernel:0' shape=(784, 10) dtype=float32, numpy=
 array([[-0.06676532, -0.02177446, -0.04767802, ..., -0.01632723,
          0.05737583,  0.06101552],
        [ 0.00299623, -0.0436253 ,  0.06276057, ..., -0.04490721,
         -0.05487836, -0.03189699],
        [-0.00015749,  0.0744667 , -0.06792812, ...,  0.01202945,
         -0.06854344,  0.06485934],
        ...,
        [-0.00544913, -0.02011172,  0.06521822, ..., -0.01148592,
          0.02759968,  0.0671545 ],
        [-0.05512473, -0.08174962,  0.0710303 , ...,  0.05021045,
          0.0841315 ,  0.08508373],
        [ 0.05080924,  0.00027304,  0.02621407, ...,  0.00100627,
          0.08438729, -0.00895975]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=
 array([-0.08289377,  0.17290778, -0.03759314, -0.05408418,  0.03751476,
         0.1621596 , -0.02009875,  0.10303649, -0.23411542, -0.04683333],
       dtype=float32)>]